In [1]:
include("XYCore.jl")
using .XYCore
include("../../logger/logger.jl")
using .MyLogger
include("../utils.jl")
using .Utils
using Logging, LinearAlgebra, Statistics, StatsBase, Random
using CairoMakie

init_logger(dir="../../logs", level=:Info)
println(names(XYCore))

[:XYCore, :calc_acceptance_and_rtt, :calc_energy, :calc_magnetization, :initial_state, :metropolis_sampler, :score_method, :wolff_sampler]


In [2]:
N = 20
β_min = 0.5
β_max = 2.0
M = 20
mcSteps=10^7
@info """
Hyperparameters of XY model:
--------------------
system size: $(N*N),
the number of replica: $M,
MCS: $mcSteps
"""
println("Set hyperparameters.")

Set hyperparameters.


In [3]:
β_ladder_trace = [[0.47608451174283384, 0.5349073876328612, 0.5920508179978864, 0.6482285148545113, 0.7029401116686586, 0.7552453038374775, 0.8070470454946345, 0.8570094449943182, 0.9052355157559144, 0.956599456247097, 1.0158025006996814, 1.081657804051085, 1.1580616307951335, 1.2458984588814268, 1.3466466568996367, 1.4577755539186685, 1.5826644083541614, 1.7197531948713185, 1.8731373192180718, 2.041366227658945]]
geometric_β_ladder = Utils.set_temperature_ladder(
    β_min, 
    β_max; 
    M=M, 
    method=:geometric
)
inverse_linear_β_ladder = Utils.set_temperature_ladder(
    β_min, 
    β_max; 
    M=M, 
    method=:inverse_linear
)
push!(β_ladder_trace, geometric_β_ladder)
push!(β_ladder_trace, inverse_linear_β_ladder)

3-element Vector{Vector{Float64}}:
 [0.47608451174283384, 0.5349073876328612, 0.5920508179978864, 0.6482285148545113, 0.7029401116686586, 0.7552453038374775, 0.8070470454946345, 0.8570094449943182, 0.9052355157559144, 0.956599456247097, 1.0158025006996814, 1.081657804051085, 1.1580616307951335, 1.2458984588814268, 1.3466466568996367, 1.4577755539186685, 1.5826644083541614, 1.7197531948713185, 1.8731373192180718, 2.041366227658945]
 [0.5000000000000004, 0.5378452931100917, 0.5785551186413602, 0.6223462947320121, 0.6694520506122364, 0.7201232687693798, 0.7746298211333282, 0.8332620063985448, 0.8963320961378562, 0.9641759979424954, 1.0371550444461923, 1.115657917761544, 1.2001027195781033, 1.2909391979474052, 1.3886511426146564, 1.493758961654486, 1.606822453133765, 1.7284437865632112, 1.8592707100168127, 2.0]
 [0.5, 0.5789473684210527, 0.6578947368421053, 0.7368421052631579, 0.8157894736842105, 0.8947368421052632, 0.9736842105263157, 1.0526315789473686, 1.131578947368421, 1.2105263157894

In [4]:
mean_rtt_trace = []
strategys = ["uniform acceptance", "geometric", "inverse linear"]

for (β_ladder, strategy) in zip(β_ladder_trace, strategys)
    exchange_prob_means, all_rtts = XYCore.calc_acceptance_and_rtt(
        N, 
        β_ladder;
        mcSteps=mcSteps
    )
    mean_rtt = mean(all_rtts)

    @info """
        $strategy: 
        inverse temperature ladder = $β_ladder, 
        Mean Round-Trip Time: $mean_rtt ± $(Statistics.std(all_rtts))"""
    println("""
        $strategy: 
        inverse temperature ladder = $β_ladder, 
        Mean Round-Trip Time: $mean_rtt ± $(Statistics.std(all_rtts))"""
    )
end

uniform acceptance: 
inverse temperature ladder = [0.47608451174283384, 0.5349073876328612, 0.5920508179978864, 0.6482285148545113, 0.7029401116686586, 0.7552453038374775, 0.8070470454946345, 0.8570094449943182, 0.9052355157559144, 0.956599456247097, 1.0158025006996814, 1.081657804051085, 1.1580616307951335, 1.2458984588814268, 1.3466466568996367, 1.4577755539186685, 1.5826644083541614, 1.7197531948713185, 1.8731373192180718, 2.041366227658945], 
Mean Round-Trip Time: 8294.887975760594 ± 4964.788182701964
geometric: 
inverse temperature ladder = [0.5000000000000004, 0.5378452931100917, 0.5785551186413602, 0.6223462947320121, 0.6694520506122364, 0.7201232687693798, 0.7746298211333282, 0.8332620063985448, 0.8963320961378562, 0.9641759979424954, 1.0371550444461923, 1.115657917761544, 1.2001027195781033, 1.2909391979474052, 1.3886511426146564, 1.493758961654486, 1.606822453133765, 1.7284437865632112, 1.8592707100168127, 2.0], 
Mean Round-Trip Time: 9282.650750150959 ± 5740.92555125168
inve